In [13]:
import zipfile
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# California

In [2]:
df_ca = pd.read_pickle("../data/ca_cleaned.pkl", compression= "gzip")

In [3]:
df_ca.head()

,date,date_dt,county_name,subject_race,subject_sex,search_conducted,search_basis,outcome
0,2009-07-01,2009-07-01,Stanislaus County,other,male,False,no_search,unknown
1,2009-07-01,2009-07-01,Stanislaus County,hispanic,female,False,no_search,summons
2,2009-07-01,2009-07-01,Stanislaus County,hispanic,female,True,other,summons
3,2009-07-01,2009-07-01,Stanislaus County,white,female,False,no_search,summons
4,2009-07-01,2009-07-01,Stanislaus County,hispanic,male,True,other,summons


In [4]:
df_ca["date"] = pd.to_datetime(df_ca["date"])

In [5]:
df_ca["year"] = df_ca["date"].dt.year

There are some spelling mistakes in county names and suffixe County to delete at the end of the string.

In [6]:
df_ca['county_name'] = df_ca['county_name'].str[:-7]
df_ca['county_name'] = df_ca['county_name'].apply(lambda x : x.replace("Siskyou", "Siskiyou"))
df_ca['county_name'] = df_ca['county_name'].apply(lambda x : x.replace("Santa Barbera", "Santa Barbara"))

## Ethnicity

In [7]:
count_year_ethnicity_ca = df_ca.groupby(["year", "county_name", "subject_race"]).count()["date"]
count_year_ethnicity_ca = pd.DataFrame(count_year_ethnicity_ca)
count_year_ethnicity_ca = count_year_ethnicity_ca.rename(columns = {"date": "nb_arrest"})

count_year_ethnicity_ca.head(10)


nb_arrest
year county_name subject_race                     
2009 Alameda     asian/pacific islander      13886
                 black                       20976
                 hispanic                    25689
                 other                       13071
                 white                       40816
     Alpine      asian/pacific islander        341
                 black                         125
                 hispanic                      661
                 other                         348
                 white                        5630

In [8]:
count_year_ethnicity_ca = count_year_ethnicity_ca.reset_index()

## Gender

In [9]:
count_year_gender_ca = df_ca.groupby(["year", "county_name", "subject_sex"]).count()["date"]
count_year_gender_ca = pd.DataFrame(count_year_gender_ca)
count_year_gender_ca = count_year_gender_ca.rename(columns = {"date": "nb_arrest"})

count_year_gender_ca.head(10)

nb_arrest
year county_name subject_sex           
2009 Alameda     female           34786
                 male             79651
     Alpine      female            1895
                 male              5210
     Amador      female            2239
                 male              4773
     Butte       female            6795
                 male             11053
     Calaveras   female            3282
                 male              6266

In [10]:
count_year_gender_ca = count_year_gender_ca.reset_index()

In [11]:
count_year_ethnicity_ca.to_pickle("../data/count_year_ethnicity_ca.pkl", compression="gzip")
count_year_gender_ca.to_pickle("../data/count_year_gender_ca.pkl", compression="gzip")


# Population

In [5]:
population_ca = pd.read_csv('../data/us_county.csv')
population_ca = population_ca.query("state == 'California'")
population_ca.drop(['fips','female_percentage','long','lat','median_age','state_code','state'], axis = 1, inplace = True)
population_ca['county'] = population_ca['county'].str[:-7]
population_ca.head()

,county,male,female,population
186,Alameda,807171,836529,1643700
187,Alpine,612,534,1146
188,Amador,20302,17527,37829
189,Butte,112513,114562,227075
190,Calaveras,22434,22801,45235


#### Ethni

In [13]:
relative_count_ethnicity_ca = pd.merge(count_year_ethnicity_ca, population_ca, left_on = 'county_name', right_on = 'county').drop(['county'], axis = 1)
relative_count_ethnicity_ca['relative_arrest'] = relative_count_ethnicity_ca['nb_arrest'] / relative_count_ethnicity_ca['population']

In [14]:
relative_count_ethnicity_ca

,year,county_name,subject_race,nb_arrest,male,female,population,relative_arrest
0,2009,Alameda,asian/pacific islander,13886,807171,836529,1643700,0.008448
1,2009,Alameda,black,20976,807171,836529,1643700,0.012761
2,2009,Alameda,hispanic,25689,807171,836529,1643700,0.015629
3,2009,Alameda,other,13071,807171,836529,1643700,0.007952
4,2009,Alameda,white,40816,807171,836529,1643700,0.024832
...,...,...,...,...,...,...,...,...
2235,2016,Yolo,asian/pacific islander,774,104375,110602,214977,0.003600
2236,2016,Yolo,black,982,104375,110602,214977,0.004568
2237,2016,Yolo,hispanic,2222,104375,110602,214977,0.010336
2238,2016,Yolo,other,1032,104375,110602,214977,0.004801


#### Gender

In [15]:
relative_count_gender_ca = pd.merge(count_year_gender_ca, population_ca, left_on = 'county_name', right_on = 'county').drop(['county'], axis = 1)

relative_count_gender_ca_female = relative_count_gender_ca.query("subject_sex == 'female'")
relative_count_gender_ca_female['relative_arrest'] = relative_count_gender_ca_female['nb_arrest'] / relative_count_gender_ca_female['female']
relative_count_gender_ca_male = relative_count_gender_ca.query("subject_sex == 'male'")
relative_count_gender_ca_male['relative_arrest'] = relative_count_gender_ca_male['nb_arrest'] / relative_count_gender_ca_male['male']
relative_count_gender_ca = pd.concat([relative_count_gender_ca_female, relative_count_gender_ca_male])

-----

# Texas

In [16]:
df_tx = pd.read_pickle("../data/tx_cleaned.pkl", compression= "gzip")
df_tx.head()

,date,date_dt,county_name,subject_race,subject_sex,search_conducted,search_basis,outcome
7647464,2009-01-01,2009-01-01,Bee County,white,female,False,no_search,warning
7647465,2009-01-01,2009-01-01,Harris County,white,male,False,no_search,warning
7647466,2009-01-01,2009-01-01,Zapata County,hispanic,male,False,no_search,citation
7647467,2009-01-01,2009-01-01,Val Verde County,hispanic,male,False,no_search,warning
7647468,2009-01-01,2009-01-01,Howard County,white,female,False,no_search,warning


In [17]:
df_tx["date"] = pd.to_datetime(df_tx["date"])
df_tx["year"] = df_tx["date"].dt.year
df_tx['county_name'] = df_tx['county_name'].str[:-7]

## Ethnicity

In [18]:
count_year_ethnicity_tx = df_tx.groupby(["year", "county_name", "subject_race"]).count()["date"]
count_year_ethnicity_tx = pd.DataFrame(count_year_ethnicity_tx)
count_year_ethnicity_tx = count_year_ethnicity_tx.rename(columns = {"date": "nb_arrest"})

count_year_ethnicity_tx.head(10)


nb_arrest
year county_name subject_race                     
2009 Anderson    asian/pacific islander         57
                 black                        1557
                 hispanic                     1029
                 other                          13
                 unknown                       148
                 white                        7235
     Andrews     asian/pacific islander         17
                 black                          97
                 hispanic                     1612
                 other                           2

In [19]:
count_year_ethnicity_tx = count_year_ethnicity_tx.reset_index()

## Gender

In [20]:
count_year_gender_tx = df_tx.groupby(["year", "county_name", "subject_sex"]).count()["date"]
count_year_gender_tx = pd.DataFrame(count_year_gender_tx)
count_year_gender_tx = count_year_gender_tx.rename(columns = {"date": "nb_arrest"})

count_year_gender_tx.head(10)

nb_arrest
year county_name subject_sex           
2009 Anderson    female            3618
                 male              6421
     Andrews     female             969
                 male              2663
     Angelina    female            4419
                 male              8365
     Aransas     female            1243
                 male              2882
     Archer      female            1918
                 male              3524

In [21]:
count_year_gender_tx = count_year_gender_tx.reset_index()

## Population

In [22]:
population_tx = pd.read_csv('../data/us_county.csv')
population_tx = population_tx.query("state == 'Texas'")
population_tx.drop(['fips','female_percentage','long','lat','median_age','state_code','state'], axis = 1, inplace = True)
population_tx['county'] = population_tx['county'].str[:-7]
population_tx.head()

,county,male,female,population
2523,Anderson,35412,22451,57863
2524,Andrews,9102,8716,17818
2525,Angelina,42659,44948,87607
2526,Aransas,12026,12737,24763
2527,Archer,4326,4463,8789


In [23]:
relative_count_ethnicity_tx = pd.merge(count_year_ethnicity_tx, population_tx, left_on = 'county_name', right_on = 'county').drop(['county'], axis = 1)
relative_count_ethnicity_tx['relative_arrest'] = relative_count_ethnicity_tx['nb_arrest'] / relative_count_ethnicity_tx['population']

In [24]:
relative_count_gender_tx = pd.merge(count_year_gender_tx, population_tx, left_on = 'county_name', right_on = 'county').drop(['county'], axis = 1)

relative_count_gender_tx_female = relative_count_gender_tx.query("subject_sex == 'female'")
relative_count_gender_tx_female['relative_arrest'] = relative_count_gender_tx_female['nb_arrest'] / relative_count_gender_tx_female['female']
relative_count_gender_tx_male = relative_count_gender_tx.query("subject_sex == 'male'")
relative_count_gender_tx_male['relative_arrest'] = relative_count_gender_tx_male['nb_arrest'] / relative_count_gender_tx_male['male']
relative_count_gender_tx = pd.concat([relative_count_gender_tx_female, relative_count_gender_tx_male])

# All

In [25]:
arrest_ethnicity = pd.concat([relative_count_ethnicity_ca, relative_count_ethnicity_tx])

In [26]:
arrest_ethnicity

,year,county_name,subject_race,nb_arrest,male,female,population,relative_arrest
0,2009,Alameda,asian/pacific islander,13886,807171,836529,1643700,0.008448
1,2009,Alameda,black,20976,807171,836529,1643700,0.012761
2,2009,Alameda,hispanic,25689,807171,836529,1643700,0.015629
3,2009,Alameda,other,13071,807171,836529,1643700,0.007952
4,2009,Alameda,white,40816,807171,836529,1643700,0.024832
...,...,...,...,...,...,...,...,...
11656,2016,Zavala,asian/pacific islander,24,6208,5923,12131,0.001978
11657,2016,Zavala,black,98,6208,5923,12131,0.008078
11658,2016,Zavala,hispanic,4845,6208,5923,12131,0.399390
11659,2016,Zavala,other,4,6208,5923,12131,0.000330


In [27]:
arrest_gender = pd.concat([relative_count_gender_ca, relative_count_gender_tx])
arrest_gender.head()

,year,county_name,subject_sex,nb_arrest,male,female,population,relative_arrest
0,2009,Alameda,female,34786,807171,836529,1643700,0.041584
2,2010,Alameda,female,76046,807171,836529,1643700,0.090907
4,2011,Alameda,female,87530,807171,836529,1643700,0.104635
6,2012,Alameda,female,74437,807171,836529,1643700,0.088983
8,2013,Alameda,female,76476,807171,836529,1643700,0.091421


In [28]:
arrest_ethnicity.to_csv('../data/arrest_ethnicity.csv')
arrest_gender.to_csv('../data/arrest_gender.csv')

# Data processing

In [15]:
path_data = "../data/ca_cleaned.pkl"
path_population = '../data/us_county.csv'

def pre_process(path):
    df = pd.read_pickle(path, compression= "gzip")
    df["date"] = pd.to_datetime(df["date"])
    df["year"] = df["date"].dt.year
    df['county_name'] = df['county_name'].str[:-7]
    return df

def count_year_one_state(df, object = 'ethnicity'):
    if object == 'ethnicity':
        count_year = df.groupby(["year", "county_name", "subject_race"]).count()["date"]
        count_year = pd.DataFrame(count_year)
        count_year = count_year.rename(columns = {"date": "nb_arrest"})
        count_year = count_year.reset_index()
    else:
        count_year = df.groupby(["year", "county_name", "subject_sex"]).count()["date"]
        count_year = pd.DataFrame(count_year)
        count_year = count_year.rename(columns = {"date": "nb_arrest"})
        count_year = count_year.reset_index()
        
    return count_year

def load_population(state):
    population = pd.read_csv('../data/us_county.csv')
    population = population.query("state == state")
    population.drop(['fips','female_percentage','long','lat','median_age','state_code','state'], axis = 1, inplace = True)
    population['county'] = population['county'].str[:-7]
    return population

def relative_arrest_computation(df_state, population, object='ethnicity'):
    if(object == 'ethnicity'):
        relative_arrest = pd.merge(df_state, population, left_on = 'county_name', right_on = 'county').drop(['county'], axis = 1)
        relative_arrest['relative_arrest'] = relative_arrest['nb_arrest'] / relative_arrest['population']
    else:
        relative_arrest = pd.merge(df_state, population, left_on = 'county_name', right_on = 'county').drop(['county'], axis = 1)
        relative_arrest_female = relative_arrest.query("subject_sex == 'female'")
        relative_arrest_female['relative_arrest'] = relative_arrest_female['nb_arrest'] / relative_arrest_female['female']
        relative_arrest_male = relative_arrest.query("subject_sex == 'male'")
        relative_arrest_male['relative_arrest'] = relative_arrest_male['nb_arrest'] / relative_arrest_male['male']
        relative_arrest = pd.concat([relative_arrest_female, relative_arrest_male])

    return relative_arrest

def create_df_one_year(path, state):
    df = pre_process(path)
    count_year_ethnicity = count_year_one_state(df)
    count_year_gender = count_year_one_state(df, object = 'gender')
    population = load_population(state)
    df_ethnicity = relative_arrest_computation(count_year_ethnicity, population)
    df_ethnicity['state'] = state
    df_gender = relative_arrest_computation(count_year_gender, population, object='gender')
    df_gender['state'] = state

    return df_ethnicity, df_gender

In [16]:
df_ethnicity_ca, df_gender_ca = create_df_one_year("../data/ca_cleaned.pkl", 'California')


In [17]:
df_ethnicity_tx, df_gender_tx = create_df_one_year("../data/tx_cleaned.pkl", 'Texas')

In [18]:
arrest_ethnicity = pd.concat([df_ethnicity_ca, df_ethnicity_tx])
arrest_gender = pd.concat([df_gender_ca, df_gender_tx])
arrest_ethnicity.to_csv('../data/arrest_ethnicity.csv')
arrest_gender.to_csv('../data/arrest_gender.csv')